<h1 style="text-align:center;">Data Wrangling</h1>

# Introduction

**Data wrangling**, often also referred to as **data munging**, is the process of cleaning, structuring, and enriching raw data into a desired format for better decision-making. It's a fundamental step in the data preparation process before analysis or processing. Data wrangling involves several tasks and can be quite complex depending on the state of the data and the desired outcome. 

Here are some steps you would follow:

1. **Cleaning**: This involves removing, correcting, or handling corrupted, misformatted, incomplete, or inaccurate data. Common tasks include handling missing values, removing duplicates, and correcting data inconsistencies.
2. **Transforming**: This involves converting data from one format or structure into another. Examples include normalizing and scaling, pivoting tables, or converting data types.
3.  **Enriching**: Enhancing data with additional variables or attributes can make the dataset more useful. This could involve adding data from other sources or creating new derived variables.
4.  **Validating**: Ensuring that the dataset meets certain criteria, often set by predefined rules or schemas. This can involve checking for data consistency, accuracy, and relevance.
5.  **Structuring**: Organizing data in a way that is suitable for the intended purpose. This could involve grouping, sorting, or aggregating data, or reshaping datasets.
6.  **Integrating**: Combining data from multiple sources, which may involve tasks like data alignment, deduplication, and dealing with data source discrepancies.

## Chaining

In this work, we will use a concept called _Chaining_. At its core, chaining is about executing multiple operations in a sequence, where the output of one operation feeds directly into the input of the next.

Think of a factory assembly line where raw materials enter one end and go through several machines, each performing a specific task, before a finished product comes out the other end. At each stage, the output of one machine becomes the input for the next.

In our case, using pandas, chaining mirrors this assembly line concept. Instead of machines, we have methods (functions associated with objects), and instead of raw material, we have data.

One thing we will spend time doing would be to create a new column or alter an old one; and there are several ways we could do this. 

```python
df['new_column'] = value_or_function

df.assign(new_column=value_or_function)
```

Both of the methods above would give us a new column with the name stated or if the column already exists, to alter the contents based off of the values assigned to it. In this walk-through, we will strictly use the `.assign` method most of the time; as this lends itself to the method of chaining well.


import pandas as pd
import datetime as dt

## Datasets: Bike Rentals

We are opting out for the bike rentals dataset from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/index.php), a world-famous data warehouse that is free to the public. The author of Hands-On Gradient Boosting with XGBoost and scikit-learn adjusted from the original dataset. We will depend on this book for our walk-through here.

In [1]:
url = 'https://raw.githubusercontent.com/theAfricanQuant/XGBoost4machinelearning/main/data/bike_rentals.csv'

In [2]:
def get_data(url):
    return (
        pd.read_csv(url)
    )

df_bikes = get_data(url)

NameError: name 'pd' is not defined

In [ ]:
df_bikes.head()

## Understanding the data

In [ ]:
df_bikes.describe()

In [ ]:
df_bikes.info()

**Correcting null values**

The following code will sum the total number of null values in the dataset. We will chain the methods and functions together horizontally.

In [ ]:
def total_nulls(df):
    return (df
         .isna()
         .sum()
         .sum()
        )

total_nulls(df_bikes)

We will now create a function that can display the the rows that have null values. In creating a function, we plan to use it over and over again in this project or any other in the future.

In [ ]:
def show_nulls(df):
    return (df[df
            .isna()
            .any(axis=1)]
           )

show_nulls(df_bikes)

We will replace the null values in the windspeed column with the median. We choose to use the median over the median because the mean tends to guarantee that half the data is greater than the given value and half the data is lower. The mean, by contrast, is vulnerable to outliers. This is the begining of building our wrangle function and we will call it `prep_data`. We will continue to build it step by step until the end of the notebook.

In [ ]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())))           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

For the 'windspeed' column, the null values may be replaced with the median value

In [ ]:
bikes.iloc[[56, 81]]

It's possible to get more nuanced when correcting null values by using a `groupby`.

A `groupby` organizes rows by shared values. Since there are four shared seasons spread out among the rows, a `groupby` of seasons results in a total of four rows, one for each season. But each season comes from many different rows with different values. We need a way to combine, or aggregate, the values. Choices for the aggregate include `.sum()`, .`count()`, .`mean()`, and .`median()`. 

Grouping our dataframe by season with the `.median(numeric_only=True)` aggregate is achieved as follows:

In [ ]:
bikes.groupby(['season']).median(numeric_only=True)

To correct the null values in the `'hum'` column (humidity), we can take the median humidity by season.
```python
bikes['hum'] = bikes['hum'].fillna()
```
The code that goes inside fillna is the desired values. The values obtained from groupby require the transform method as follows:

```python
bikes.groupby('season')['hum'].transform('median')
```
Bringing everything together:

```python
bikes['hum'] = (bikes['hum']
                   .fillna(bikes.groupby('season')['hum']
                           .transform('median'))
                  )
```

However, to implement it we are going to use the method of chaining and the `.assign` method in our code.

In [ ]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))
                  )
                   )           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

from the above we can see that the column `'hum'` has been taken off the those with `'nan'` values.

In [ ]:
bikes.iloc[[129, 213, 388]]

**Obtaining the median/mean from specific rows**

In some cases, it may be advantageous to replace null values with data from specific rows.

When correcting temperature, aside from consulting historical records, taking the mean temperature of the day before and the day after should give a good estimate. 

To find null values of the 'temp' column, enter the following code:

In [ ]:
bikes[bikes['temp'].isna()]

Index `701` contains null values.

We will now find the mean temperature of the day before and the day after the 701 index, using the some steps:

1. Let us sum the temperatures in rows `700` and `702` and divide by `2` for both the `'temp'` and `'atemp'` columns

In [ ]:
def mean_vals(df, idx1, idx2, col):
    return (
        (df.iloc[idx1][col] + 
        df.iloc[idx2][col])/2
    )

mean_vals(bikes, 700, 702, 'atemp')

In [ ]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp')))
                   )           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

**Extrapolate dates**

`'dteday'` is meant to be a date column but the `.info` we ran earlier revealed to us that it was an object or a string. Date objects such as years and months must be extrapolated from `datetime` types. Lets convert the column to a `datetime`.

In [ ]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday'])
                   )           
           )

bikes = (prep_data(df_bikes)
        .info())

We have imported the `'datetime'` library above. 

We will convert the `'mnth'` column to the correct months extrpolated from the `'dteday'` column.

In [ ]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday']),
                    mnth = lambda x: x['dteday'].dt.month
                   )           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

lets us check the last 5 values of the dataset we have worked on so far.

In [ ]:
bikes.tail(5)

We can see that even though the year value on the `'dteday'` column has 2012 all through, the value on the `'yr'` column is `1.0`. It probably means that the values have been normalized, probably between 0 & 1. I think this was done because normalized data is often more efficient due to the fact that machine learning weights do not have to adjust for different ranges.

We will just use the forward fill for the null values here since the row with the null value is in the same month with the preceding row.

```python
data['yr'].ffill()
```

or 

```python
data['yr'].fillna(method='ffill')
```

In [ ]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday']),
                    mnth = lambda x: x['dteday'].dt.month,
                    yr = data['yr'].ffill()
                   )           
           )

bikes = prep_data(df_bikes)
show_nulls(bikes)

There are no more null values in our dataset.

In [ ]:
total_nulls(bikes)

In [ ]:
bikes.tail()

**Deleting non-numerical columns**

For machine learning, all data columns should be numerical. According to `df.info()`, the only column that is not numerical is `'dteday'`. Furthermore, it's redundant since all date information exists in other columns.

In [ ]:
def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday']),
                    mnth = lambda x: x['dteday'].dt.month,
                    yr = data['yr'].ffill()
                   )
            .drop('dteday', axis=1)
           )

bikes = prep_data(df_bikes)
bikes.head()

### Final Code

The next thing we will do is to create a module with the functions that we have created so we could always call them whenever we need them.

In [ ]:
%%writefile data_wrangle.py

import pandas as pd
import datetime as dt


def show_nulls(df):
    return (df[df
            .isna()
            .any(axis=1)]
           )
    
def total_nulls(df):
    return (df
         .isna()
         .sum()
         .sum()
        )

def get_data(url):
    return (
        pd.read_csv(url)
    )

def mean_vals(df, idx1, idx2, col):
    return (
        (df.iloc[idx1][col] + 
        df.iloc[idx2][col])/2
    )

def prep_data(data):
    return (data
            .assign(windspeed = data["windspeed"]
                    .fillna((data["windspeed"]
                             .median())),
                    hum = (data['hum']
                   .fillna(data.groupby('season')['hum']
                           .transform('median'))),
                    temp = (data['temp']
                            .fillna(mean_vals(data, 700, 702, 'temp'))),
                    atemp = (data['atemp']
                            .fillna(mean_vals(data, 700, 702, 'atemp'))),
                    dteday = pd.to_datetime(data['dteday']),
                    mnth = lambda x: x['dteday'].dt.month,
                    yr = data['yr'].ffill()
                   )
            .drop('dteday', axis=1)
           )